## Segmenting and Clustering Neighborhoods in Toronto

#### 1. Scrape the data to pandas Dataframe from the following wikipedia page:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

####  To create the above dataframe:

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [24]:
!pip install lxml
import lxml
import pandas as pd
import numpy as np
print('Libraries improted')

Libraries improted


#### Use "read_html" to scrape the table

In [25]:
# dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
# df = dfs[0]
# df = df[df.Borough != 'Not assigned']
# df = df.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
# df.shape

In [26]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url, header=0)
table = tables[0]
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [27]:
table.shape

(180, 3)

#### Check the number of "Not assigned" in Borough and Neighborhood

In [28]:
table.Borough.value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [29]:
table.Neighbourhood.value_counts()

Not assigned                                                                                                                              77
Downsview                                                                                                                                  4
Don Mills                                                                                                                                  2
The Annex, North Midtown, Yorkville                                                                                                        1
Runnymede, Swansea                                                                                                                         1
Rouge Hill, Port Union, Highland Creek                                                                                                     1
Parkdale, Roncesvalles                                                                                                                     1
Parkview Hill

#### Use nan to replace "Not assigned" in Borough.

In [30]:
table.Borough.replace("Not assigned", np.nan, inplace = True)
table.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop the rows where borough has Nan.

In [31]:
table.dropna(axis = 0, inplace=True)
table = table.reset_index()
table = table.drop(['index'],axis = 1)
table.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Group Neighbourhoods with the same Postcode

In [42]:
table = table.groupby(['Postal Code', 'Borough'])['Neighbourhood'].agg(lambda x:'%s' % ','.join(x))
table = table.reset_index()
table.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [38]:
# No "Not Assigned" neibourhood anymore.
table.Neighbourhood.value_counts()

Downsview                                                                                                                                 4
Don Mills                                                                                                                                 2
Central Bay Street                                                                                                                        1
Lawrence Park                                                                                                                             1
Weston                                                                                                                                    1
Wexford, Maryvale                                                                                                                         1
Bayview Village                                                                                                                           1
Rosedale            

In [39]:
table.shape
print("The number of rows:", table.shape[0])

The number of rows: 103


#### To scape the data with BeautifulSoup

In [47]:
pip install beautifulsoup4

     |████████████████████████████████| 122kB 5.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [48]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [49]:
List_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(List_url).text

In [50]:
soup = BeautifulSoup(source,'xml')

In [51]:
table=soup.find('table')

In [52]:
#dataframe will consist of three columns: PostalCode, Botough and Neighborhood
column_names = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(columns = column_names)

In [53]:
# Search all the postcode, borough, neighborhood
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
        if len(row_data)==3:
            df.loc[len(df)]=row_data
    

In [54]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Data Cleaning
* remove rows where Borough is "Not assigned"

In [55]:
df=df[df['Borough']!='Not assigned']

In [66]:
# There are not 'Neighborhood' named as 'Not assigned' anymore.
df[df['Neighborhood']=='Not assigned']


,PostalCode,Borough,Neighborhood


In [68]:
print("The number of rows:", df.shape[0])

The number of rows: 103
